In [3]:
###
import csv
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import mplleaflet
import networkx as nx
import branca.colormap as cm
from scipy.spatial import distance
from numpy import array
from numpy.linalg import norm
from scipy.optimize import minimize

#####################################
columns_signal = defaultdict(list) 

with open('PE_pluviometria.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        for (i,v) in enumerate(row):
            columns_signal[i].append(v)
            
l = len(columns_signal[0])
#Data in columns_signal[3]

columns_coordinates = defaultdict(list) 

with open('Coordenadas_das_estacoes.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        for (i,v) in enumerate(row):
            columns_coordinates[i].append(v)
            
k = len(columns_coordinates[0])
#Coordinates in columns_coordinates[2] and columns_coordinates[3]

#####################################
Data_matrix = np.zeros((l,3))

for j in range (0,l):
    if (columns_signal[3][j] != '-'):
        Data_matrix[j][0] = float(columns_coordinates[2][j])
        Data_matrix[j][1] = float(columns_coordinates[3][j])
        Data_matrix[j][2] = float(columns_signal[3][j])
        
Data_matrix = Data_matrix[~np.all(Data_matrix == 0, axis=1)]

coordinates = np.zeros((len(Data_matrix),2))
signal = np.zeros((len(Data_matrix),1))

for i in range(0,len(Data_matrix)):
    coordinates[i][1] = Data_matrix[i][0]
    coordinates[i][0] = Data_matrix[i][1]
    signal[i] = Data_matrix[i][2]
    
#####################################
def m_adj(coords,k):
    
    Adj_K = np.zeros((len(coords),len(coords)))
    Aux = np.zeros((len(coords),1))
    aux = 0

    ind=np.zeros((k,1))
    for x in range(0,len(coords)):
        for y in range(0,len(coords)):
            v = [coords[x][0],coords[x][1]]
            w = [coords[y][0],coords[y][1]]
            dst = distance.euclidean(v,w)
            Aux[y]=dst
        ind = sorted(Aux)[1:k+1]
        for i in range(0,k):
            ind[i] = np.where(Aux == ind[i])
        for j in range(0,k):
            aux = ind[j][0]
            Adj_K[x][int(aux)] = np.exp((-Aux[ind[j]]**2))
            Adj_K[int(aux)][x] = Adj_K[x][int(aux)]

    
    return(Adj_K)

A = m_adj(coordinates,6)

#####################################
ind_ = [3,5,7,8,9,12,18,25,33,36,42,46,51,53,57,59,97,112,114,116,118,123,124,126,129,132,134,137,145,154,157,159,162,163,165,167,169]

#####################################
def L(Adj):
    
    degree = np.zeros((len(Adj),len(Adj))) 

    colsum = Adj.sum(axis=0)
    rowsum = Adj.sum(axis=1)

    for j in range(0, len(Adj)):
        degree[j][j] = colsum[j] + rowsum[j]

    L = degree - Adj
    
    return(L)

Lp = L(A) 

#####################################
s_est = []

def fun(n):
    
    sig = []
    
    for j in range (0,len(signal)):
        sig.append(signal[j])
        
    global s_est
    
    for i in range(len(n)):
        sig[ind_[i]] = n[i]
            
    s_est = sig
    R = array(np.dot(Lp, s_est))
    return(norm(R))

#####################################
x0 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
res = minimize(fun, x0, method='BFGS', options={'gtol': 1e-6, 'disp': True})
res.x

#####################################
G=nx.Graph()

lst = []

for i in range(0,len(signal)):
    lst.append(i)

G.add_nodes_from(lst)

#####################################
pos = []

for i in range(0,len(signal)):
    pos.append((coordinates[i][0],coordinates[i][1]))

edges = []

for i in range(0,len(A)):
    for j in range(0,len(A)):
        if A[i][j] > 0:
            edges.append((i,j))

pos_edges = []

for i in range(0,len(A)):
    pos_edges.append(((coordinates[edges[i][0]],coordinates[edges[i][0]]),(coordinates[edges[i][1]],coordinates[edges[i][1]])))
            
G.add_edges_from(edges)

#####################################
color = []

a = cm.linear.RdBu_04.scale(0, 280)

for i in range(0,len(coordinates)):
    color.append(a.rgb_hex_str(s_est[i]))
#If you change 'signal' by 's_est' you can draw the estimated graph signal 

#####################################
fig, ax = plt.subplots()

nx.draw_networkx_nodes(G,pos=pos,node_size=10,node_color=color,edge_color='k',alpha=1, with_labels=True)
nx.draw_networkx_edges(G,pos=pos,edge_color='black', alpha=.5)
nx.draw_networkx_labels(G,pos, label_pos =10.3)

#mplleaflet.show(fig=ax.figure,tiles='mapbox bright')
#mplleaflet.show(fig=ax.figure)
mplleaflet.display(fig=ax.figure)

         Current function value: 10368.629656
         Iterations: 89
         Function evaluations: 4252
         Gradient evaluations: 109


/home/default/.local/lib/python3.5/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):
